In [1]:
import pandas as pd

In [2]:
submission = pd.read_csv("/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv")

In [3]:
years = []
teama = []
teamb = [] 
for _,row in submission.iterrows():
    years.append(int(row["ID"].split("_")[0]))
    teama.append(int(row["ID"].split("_")[1]))
    teamb.append(int(row["ID"].split("_")[2]))
submission["Season"] = years
submission["TeamA"] = teama
submission["TeamB"] = teamb

In [4]:
submission

,ID,Pred,Season,TeamA,TeamB
0,2016_1112_1114,0.5,2016,1112,1114
1,2016_1112_1122,0.5,2016,1112,1122
2,2016_1112_1124,0.5,2016,1112,1124
3,2016_1112_1138,0.5,2016,1112,1138
4,2016_1112_1139,0.5,2016,1112,1139
...,...,...,...,...,...
11385,2021_1452_1457,0.5,2021,1452,1457
11386,2021_1452_1458,0.5,2021,1452,1458
11387,2021_1455_1457,0.5,2021,1455,1457
11388,2021_1455_1458,0.5,2021,1455,1458
